In [1]:
# Initiate the spark session

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf, SparkContext
sc = SparkContext()


In [10]:
# Import essential packages into python

import os
import pyspark
import random
from numpy import array
from math import sqrt

# sc.setCheckpointDir('checkpoint')

spark = SparkSession.builder.appName('Recommendations').getOrCreate()

# Retrieve the essential spark packages
from numpy import array
from math import sqrt
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col, explode


In [7]:
# import the csv into spark

movies = spark.read.csv("/Users/magimac/Documents/Cloud Computation/Homework Assignments/Homework 2/movies.csv", header = True)
ratings = spark.read.csv("/Users/magimac/Documents/Cloud Computation/Homework Assignments/Homework 2/ratings.csv", header = True)


In [11]:
# changing variables from strings to integer and floats.

ratings = ratings.\
    withColumn('userID', col('userID').cast('integer')).\
    withColumn('MovieID', col('MovieID').cast('integer')).\
    withColumn('rating', col('rating').cast('float')).\
    drop('timestamp')
ratings.show()


+------+-------+------+
|userID|MovieID|rating|
+------+-------+------+
|     1|   1193|   5.0|
|     1|    661|   3.0|
|     1|    914|   3.0|
|     1|   3408|   4.0|
|     1|   2355|   5.0|
|     1|   1197|   3.0|
|     1|   1287|   5.0|
|     1|   2804|   5.0|
|     1|    594|   4.0|
|     1|    919|   4.0|
|     1|    595|   5.0|
|     1|    938|   4.0|
|     1|   2398|   4.0|
|     1|   2918|   4.0|
|     1|   1035|   5.0|
|     1|   2791|   4.0|
|     1|   2687|   3.0|
|     1|   2018|   4.0|
|     1|   3105|   5.0|
|     1|   2797|   4.0|
+------+-------+------+
only showing top 20 rows



In [12]:
# Join both the data frames to add movie data into ratings data frame so we can see everything together

movie_ratings = ratings.join(movies, ['movieId'], 'left')
movie_ratings.show()


+-------+------+------+--------------------+--------------------+
|MovieID|userID|rating|               Title|              Genres|
+-------+------+------+--------------------+--------------------+
|   1193|     1|   5.0|One Flew Over the...|               Drama|
|    661|     1|   3.0|James and the Gia...|Animation|Childre...|
|    914|     1|   3.0| My Fair Lady (1964)|     Musical|Romance|
|   3408|     1|   4.0|Erin Brockovich (...|               Drama|
|   2355|     1|   5.0|Bug's Life, A (1998)|Animation|Childre...|
|   1197|     1|   3.0|Princess Bride, T...|Action|Adventure|...|
|   1287|     1|   5.0|      Ben-Hur (1959)|Action|Adventure|...|
|   2804|     1|   5.0|Christmas Story, ...|        Comedy|Drama|
|    594|     1|   4.0|Snow White and th...|Animation|Childre...|
|    919|     1|   4.0|Wizard of Oz, The...|Adventure|Childre...|
|    595|     1|   5.0|Beauty and the Be...|Animation|Childre...|
|    938|     1|   4.0|         Gigi (1958)|             Musical|
|   2398| 

In [15]:
vecAssembler = VectorAssembler(inputCols=['MovieID', 'userID', 'rating'], outputCol="features")
new_df = vecAssembler.transform(movie_ratings)
new_df.show()

+-------+------+------+--------------------+--------------------+----------------+
|MovieID|userID|rating|               Title|              Genres|        features|
+-------+------+------+--------------------+--------------------+----------------+
|   1193|     1|   5.0|One Flew Over the...|               Drama|[1193.0,1.0,5.0]|
|    661|     1|   3.0|James and the Gia...|Animation|Childre...| [661.0,1.0,3.0]|
|    914|     1|   3.0| My Fair Lady (1964)|     Musical|Romance| [914.0,1.0,3.0]|
|   3408|     1|   4.0|Erin Brockovich (...|               Drama|[3408.0,1.0,4.0]|
|   2355|     1|   5.0|Bug's Life, A (1998)|Animation|Childre...|[2355.0,1.0,5.0]|
|   1197|     1|   3.0|Princess Bride, T...|Action|Adventure|...|[1197.0,1.0,3.0]|
|   1287|     1|   5.0|      Ben-Hur (1959)|Action|Adventure|...|[1287.0,1.0,5.0]|
|   2804|     1|   5.0|Christmas Story, ...|        Comedy|Drama|[2804.0,1.0,5.0]|
|    594|     1|   4.0|Snow White and th...|Animation|Childre...| [594.0,1.0,4.0]|
|   

In [17]:


kmeans = KMeans(k = 2, seed = 1)  # 2 clusters here
#model = kmeans.fit(new_df.select('features'))


TypeError: KMeans() takes no arguments